In [ ]:
import os
import pandas as pd
import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import specgram
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.checkpoint as cp
from torch import Tensor
import torchvision
from torchvision import datasets, transforms, models
import torch.optim as optim
from tqdm import tqdm
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from PIL import Image
import cv2
import random
# import torchaudio

Error: Kernel is dead

In [2]:
def addAWGN(signal, num_bits=16, augmented_num=2, snr_low=15, snr_high=30): 
    signal_len = len(signal)
    # Generate White Gaussian noise
    noise = np.random.normal(size=(augmented_num, signal_len))
    # Normalize signal and noise
    norm_constant = 2.0**(num_bits-1)
    signal_norm = signal / norm_constant
    noise_norm = noise / norm_constant
    # Compute signal and noise power
    s_power = np.sum(signal_norm ** 2) / signal_len
    n_power = np.sum(noise_norm ** 2, axis=1) / signal_len
    # Random SNR: Uniform [15, 30] in dB
    target_snr = np.random.randint(snr_low, snr_high)
    # Compute K (covariance matrix) for each noise 
    K = np.sqrt((s_power / n_power) * 10 ** (- target_snr / 10))
    K = np.ones((signal_len, augmented_num)) * K  
    # Generate noisy signal
    return signal + K.T * noise

In [36]:
X, sample_rate = librosa.load('aihub/5e2aefca5807b852d9e021e0.wav', res_type='kaiser_fast',sr=16000,offset=0.5)
addAWGN(X)[0].shape
X.shape

(69142,)

In [ ]:
labels=[]
for i in range(len(sw_list)):
    
                if i[9:10] =='a':
                    label = 0
                elif i[9:10] =='n':
                    label = 1
                elif i[9:10] =='s':
                    label = 2
                elif i[9:10] =='h':
                    label = 3

                labels.append(label)

Error: Kernel is dead

In [5]:
mylist_1= os.listdir('aihub/')
type(mylist_1)
mylist=[]
for item in mylist_1:
    if item[-3:]=='wav':
        mylist.append(item)
df = pd.read_csv('./aihub/AIhub.csv')
df

,path,emotion
0,./4y1/5e367ec8ee8206179943c051.wav,angry
1,./4y1/5e37d20fc8c25f16cd145708.wav,angry
2,./4y1/5e36610e7995ef170fc0e868.wav,angry
3,./4y1/5e41317c0b3f160367c2075c.wav,angry
4,./4y1/5e426f780b3f160367c2097d.wav,angry
...,...,...
2190,./5y2/5f7124ddf8fac448cc0a6229.wav,sadness
2191,./4y1/5e33ce7b5807b852d9e0580f.wav,sadness
2192,./4y1/5e43a429189842034d9f77a7.wav,sadness
2193,./4y1/5e44f7f0f2fe7a3b1d544784.wav,sadness


In [43]:
#wav2spec
class ETRIDataset_spectram_ai():
    def __init__(self, dataframe, frame_length=0.025, frame_stride=0.010):
        self.dataframe = dataframe
        self.frame_length = frame_length
        self.frame_stride = frame_stride
    
    def __len__(self):
        return len(self.dataframe)*3

    def __getitem__(self,index):
        if index<len(self.dataframe):
            audio_path = self.dataframe['path'][index]
            X, sample_rate = librosa.load('aihub/'+audio_path[6:], res_type='kaiser_fast',sr=16000,offset=0.5)
        elif index < len(self.dataframe)*2:
            audio_path = self.dataframe['path'][index-len(self.dataframe)]
            X, sample_rate = librosa.load('aihub/'+audio_path[6:], res_type='kaiser_fast',sr=16000,offset=0.5)
            X = addAWGN(X)[0]
        else:
            audio_path = self.dataframe['path'][index-len(self.dataframe)*2]
            X, sample_rate = librosa.load('aihub/'+audio_path[6:], res_type='kaiser_fast',sr=16000,offset=0.5)
            X = addAWGN(X)[1]
            
        sample_rate = sample_rate
        input_nfft = int(round(sample_rate*self.frame_length))
        input_stride = int(round(sample_rate*self.frame_stride))

        S = librosa.feature.melspectrogram(y=X, n_mels=64, n_fft=input_nfft, hop_length=input_stride)
        P = librosa.power_to_db(S, ref=np.max)
        df = pd.read_csv('./aihub/AIhub.csv')

        ## get label
        if index < len(self.dataframe):
            if audio_path[-3:] == 'wav':
                if df['emotion'][index] == 'angry':
                    label = 0
                elif df['emotion'][index] == 'neutral':
                    label = 1
                elif df['emotion'][index] == 'sadness':
                    label = 2
                elif df['emotion'][index] == 'happiness':
                    label = 3
            else:
                label=None
        elif index < len(self.dataframe)*2:
            if audio_path[-3:] == 'wav':
                if df['emotion'][index-len(self.dataframe)] == 'angry':
                    label = 0
                elif df['emotion'][index-len(self.dataframe)] == 'neutral':
                    label = 1
                elif df['emotion'][index-len(self.dataframe)] == 'sadness':
                    label = 2
                elif df['emotion'][index-len(self.dataframe)] == 'happiness':
                    label = 3
            else:
                label=None
        else:
            if audio_path[-3:] == 'wav':
                if df['emotion'][index-len(self.dataframe)*2] == 'angry':
                    label = 0
                elif df['emotion'][index-len(self.dataframe)*2] == 'neutral':
                    label = 1
                elif df['emotion'][index-len(self.dataframe)*2] == 'sadness':
                    label = 2
                elif df['emotion'][index-len(self.dataframe)*2] == 'happiness':
                    label = 3
            else:
                label=None
        return P, label

In [44]:
ETRIDataset_spectram_ai(df).__getitem__(5001)[0].shape

(64, 463)

In [46]:
# train,test,val split
train_size = int(0.7*len(df)*3)

val_size = int(0.2*len(df)*3)
test_size = int(len(df)*3-train_size-val_size)
train_set, val_set, test_set = torch.utils.data.random_split(ETRIDataset_spectram_ai(df),[train_size,val_size,test_size])
print(train_size)
print(val_size)
print(test_size)

4609
1317
659


In [47]:
# spec2img
def getimg(dataset,feature):
        img_path=[]
        labels=[]
        for i in range(len(dataset)):
                fig = plt.figure()
                ax = fig.add_subplot(111)
                p = librosa.display.specshow(dataset[i][0],ax=ax, sr=16000, hop_length=int(round(16000*0.025)), x_axis='time',y_axis='linear')
                extent = ax.get_window_extent().transformed(fig.dpi_scale_trans.inverted())

                fig.savefig('./image_aihub/%s_%d_%s.jpg' % (dataset[i][1],i,feature), bbox_inches=extent)
                img_path.append('./image_aihub/%s_%d_%s.jpg' % (dataset[i][1],i,feature))
                labels.append(dataset[i][1])
                plt.ioff()
                plt.close()
        return img_path , labels


In [48]:
train_path , train_labels = getimg(train_set,'train')
val_path , val_labels = getimg(val_set,'val')
test_path , test_labels = getimg(test_set,'test')

In [49]:
train_transforms = transforms.Compose([transforms.RandomRotation(30),
                                       transforms.RandomResizedCrop(224),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406],
                                                            [0.229, 0.224, 0.225])])
test_transforms = transforms.Compose([transforms.Resize(255),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406],
                                                           [0.229, 0.224, 0.225])])
                                                           

In [50]:
class img2tensor():
    def __init__(self,data_path,labels,transforms):
        self.data_path = data_path
        self.labels = labels
        self.transforms = transforms

    def __len__(self):
        return len(self.data_path)

        
    def __getitem__(self, index):
        img_path = self.data_path[index]
        image = Image.open(img_path)
        I = train_transforms(image)
        label = self.labels[index]

        return I, label
        
        

In [51]:
# set batch_size
batch_size = 16
# dataloader
train_dataloader = torch.utils.data.DataLoader(img2tensor(train_path,train_labels,train_transforms), batch_size=batch_size, shuffle=True)
val_dataloader = torch.utils.data.DataLoader(img2tensor(val_path,val_labels,test_transforms), batch_size=batch_size, shuffle=False)
dataloaders_dict ={'train':train_dataloader, 'val': val_dataloader}
# test
batch_iterator = iter(dataloaders_dict['train'])
inputs, labels = next(batch_iterator)
print(inputs.size())
print(labels)

torch.Size([16, 3, 224, 224])
tensor([1, 1, 1, 2, 1, 3, 1, 3, 1, 3, 1, 3, 0, 2, 0, 0])


In [70]:
model = torchvision.models.densenet121(pretrained=False)
model.to(torch.device('cuda'))
model.classifier = nn.Linear(in_features=1024, out_features=4)
criterion = nn.CrossEntropyLoss()
optimizer = optim.RMSprop(model.parameters() ,lr=0.00001, weight_decay=1e-6, momentum=0.9)
model.train()

DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu

In [71]:
def train(net, dataloader, criterion, optimizer, num_epochs):
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    print(device)
    model.to(device)
    torch.backends.cudnn.benchmark = True

    
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch+1,num_epochs))
        print('-------------------------------')

        for phase in ['train','val']:
            if phase == 'train':
                net.train()
            else:
                net.eval()
            epoch_loss = 0.0
            epoch_corrects = 0

            if (epoch == 0) and(phase == 'train'):
                continue
            for inputs, labels in tqdm(dataloader[phase]):
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = net(inputs)
                    loss = criterion(outputs, labels)
                    _, preds = torch.max(outputs,1)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                    epoch_loss += loss.item() *inputs.size(0)

                    epoch_corrects += torch.sum(preds == labels.data)

                    epoch_loss = epoch_loss / len(dataloader[phase].dataset)
                    epoch_acc = epoch_corrects.double() / len(dataloader[phase].dataset)


            print('{} Loss: {:.4f} ACC {:.4f}'.format(
                phase, epoch_loss, epoch_acc))
                

In [72]:
train(model, dataloaders_dict, criterion,optimizer, num_epochs=200)

cuda:0
Epoch 1/200
-------------------------------


100%|██████████| 83/83 [00:04<00:00, 17.41it/s]


val Loss: 0.0054 ACC 0.2620
Epoch 2/200
-------------------------------


100%|██████████| 289/289 [00:37<00:00,  7.73it/s]


train Loss: 0.0003 ACC 0.3582


...

val Loss: 0.0026 ACC 0.7722
Epoch 200/200
-------------------------------


100%|██████████| 289/289 [00:33<00:00,  8.65it/s]


train Loss: 0.0004 ACC 0.8525


100%|██████████| 83/83 [00:04<00:00, 18.90it/s]

val Loss: 0.0009 ACC 0.7472


In [73]:
# save weight
save_path = './saved_models/DenseNet121_img_aihub+augmentaion.pth'
torch.save(model.state_dict(),save_path)

In [74]:
# load weight
load_path = './saved_models/DenseNet121_img_aihub+augmentaion.pth'
load_weights = torch.load(load_path, map_location={'cuda:0': 'cpu'})
model = torchvision.models.densenet121(pretrained=True)
# first_conv_layer = [nn.Conv2d(1, 3, kernel_size=3, stride=1, padding=1, dilation=1, groups=1, bias=True)]
# first_conv_layer.extend(list(model.features))  
# model.features= nn.Sequential(*first_conv_layer )  
model.classifier = nn.Linear(in_features=1024, out_features=4)
criterion = nn.CrossEntropyLoss()
optimizer = optim.RMSprop(model.parameters() ,lr=0.00001, weight_decay=1e-6, momentum=0.9)
model.eval()
model.load_state_dict(load_weights)

<All keys matched successfully>

In [75]:
class Predictor(object):
    def __init__(self, model, device ='cpu',  fp16=False ):
        self.model = model
        
        self.cls_name = {0:'angry', 1:'happy', 2:'sad', 3:'neutral'}
        self.device = device
        

    def predict(self, audio):
        
        audio_info = audio
        
        outputs = self.model(audio_info)
        probability = torch.softmax(outputs,1)
        probability = probability.squeeze()
        proba, idx = torch.max(probability, dim=0)
        emo_proba = proba.item()
        print(emo_proba)
        idx = idx.item()
        emo_label = self.cls_name[idx]
        print(emo_label)
        return emo_label



In [1]:
def swdata(emotion):
    swlist_1 = os.listdir('swdata/%s/' % emotion)
    type(swlist_1)
    swlist = []
    for item in swlist_1:
        if item[-3:] == 'wav':
            swlist.append('./swdata/%s/' % emotion + item)
    return swlist

In [4]:
sw_list = swdata('angry') + swdata('happy') + swdata('sad') + swdata('neutral')

random.shuffle(sw_list)

In [78]:
#wav2spec
class swdata_spectram():
    def __init__(self, dataframe, frame_length=0.025, frame_stride=0.010):
        self.dataframe = dataframe
        self.frame_length = frame_length
        self.frame_stride = frame_stride
    
    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self,index):
        audio_path = self.dataframe[index]
        X, sample_rate = librosa.load(audio_path, res_type='kaiser_fast',sr=16000,offset=0.0)
        sample_rate = sample_rate
        input_nfft = int(round(sample_rate*self.frame_length))
        input_stride = int(round(sample_rate*self.frame_stride))

        S = librosa.feature.melspectrogram(y=X, n_mels=64, n_fft=input_nfft, hop_length=input_stride)
        P = librosa.power_to_db(S, ref=np.max)
        

        ## get label
        if audio_path[-3:] == 'wav':
            if audio_path[9:10] =='a':
                label = 0
            elif audio_path[9:10] =='n':
                label = 1
            elif audio_path[9:10] =='s':
                label = 2
            elif audio_path[9:10] =='h':
                label = 3
        else:
            label=None
        return P, label

In [79]:
test_swdata = swdata_spectram(sw_list)
test_path , test_labels = getimg(test_swdata,'test_sw')


In [80]:
test_path

['./image_aihub/0_0_test_sw.jpg',
 './image_aihub/2_1_test_sw.jpg',
 './image_aihub/0_2_test_sw.jpg',
 './image_aihub/1_3_test_sw.jpg',
 './image_aihub/0_4_test_sw.jpg',
 './image_aihub/3_5_test_sw.jpg',
 './image_aihub/3_6_test_sw.jpg',
 './image_aihub/2_7_test_sw.jpg',
 './image_aihub/3_8_test_sw.jpg',
 './image_aihub/2_9_test_sw.jpg',
 './image_aihub/2_10_test_sw.jpg',
 './image_aihub/3_11_test_sw.jpg',
 './image_aihub/2_12_test_sw.jpg',
 './image_aihub/0_13_test_sw.jpg',
 './image_aihub/1_14_test_sw.jpg',
 './image_aihub/1_15_test_sw.jpg',
 './image_aihub/2_16_test_sw.jpg',
 './image_aihub/0_17_test_sw.jpg',
 './image_aihub/1_18_test_sw.jpg',
 './image_aihub/3_19_test_sw.jpg',
 './image_aihub/1_20_test_sw.jpg',
 './image_aihub/0_21_test_sw.jpg',
 './image_aihub/1_22_test_sw.jpg',
 './image_aihub/0_23_test_sw.jpg',
 './image_aihub/3_24_test_sw.jpg',
 './image_aihub/2_25_test_sw.jpg',
 './image_aihub/1_26_test_sw.jpg',
 './image_aihub/1_27_test_sw.jpg',
 './image_aihub/1_28_test_sw.j

In [81]:
img2tensor(test_path,test_labels,test_transforms)[1][0].unsqueeze(0).shape

torch.Size([1, 3, 224, 224])

In [82]:
predictor = Predictor(model)
device='cpu'
a=[]
b=[]
for i in range(len(img2tensor(test_path,test_labels,test_transforms))):
    a.append(predictor.predict(img2tensor(test_path,test_labels,test_transforms)[i][0].unsqueeze(0)))
    b.append(img2tensor(test_path,test_labels,test_transforms)[i][1])


0.4588371813297272
happy
0.9975912570953369
sad
0.5896168947219849
angry
0.9999948740005493
happy
0.9266456365585327
angry
0.9815202355384827
neutral
0.9962092638015747
angry
0.9999806880950928
happy
0.4997401535511017
sad
0.9901108145713806
happy
0.7102181911468506
happy
0.5159084796905518
angry
0.3106321096420288
sad
0.8508530855178833
neutral
0.9425170421600342
happy
0.3338918387889862
angry
0.5259073972702026
neutral
0.7669328451156616
happy
0.9751814007759094
happy
0.9032300114631653
happy
0.999714195728302
happy
0.9467054009437561
happy
0.9985320568084717
happy
0.8597847819328308
angry
0.9802824258804321
happy
0.6439152956008911
angry
0.9993358254432678
happy
0.916290283203125
happy
0.8401391506195068
happy
0.5600963830947876
happy
0.8829940557479858
happy
0.44337546825408936
sad
0.9985812902450562
happy
0.9505743980407715
happy
0.3510867655277252
angry
0.6103296279907227
sad
0.8476259112358093
neutral
0.8677766919136047
happy
0.7601339817047119
happy
0.7126401662826538
happy
0.9

In [83]:
df = pd.DataFrame(a,columns=['predict'])
df['original']=b
for i in range(len(df)):
    if df['original'][i] == 0:
        df['original'][i] = 'angry'
    elif df['original'][i] == 1:
        df['original'][i] = 'happy'
    elif df['original'][i] == 2:
        df['original'][i] = 'sad'
    elif df['original'][i] == 3:
        df['original'][i] = 'neutral'
print('accuracy={}'.format((df['predict']==df['original']).sum()/len(df)))

accuracy=0.39325842696629215


/tmp/ipykernel_6607/4207275683.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['original'][i] = 'angry'
/home/seojungin/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [84]:
df.to_csv('predict_result_img_sw_aihub+augmentaion.csv',index=False)
